In [5]:
import requests
import json
import pandas as pd

def fetch_resumes(vacancy, area_id, per_page=10, auth_token=None):
    """
    Fetch resumes from HH.ru API.
    Args:
        vacancy (str): Desired vacancy title.
        area_id (int): Area ID from HH API (e.g., 1 for Moscow).
        per_page (int): Number of results per page.
        auth_token (str): Optional. OAuth token for HH API.

    Returns:
        pd.DataFrame: DataFrame with resumes information or None if error occurs.
    """
    url = "https://api.hh.ru/resumes"
    headers = {
        "User-Agent": "ResumeFetcher/1.0 (myemail@example.com)"
    }
    if auth_token:
        headers["Authorization"] = f"Bearer {auth_token}"

    params = {
        "text": vacancy,
        "area": area_id,
        "per_page": per_page,
    }

    response = requests.get(url, headers=headers, params=params)

    if response.status_code == 403:
        print("Error fetching resumes: 403 Forbidden. Check your token or permissions.")
        return None
    elif response.status_code != 200:
        print(f"Error fetching resumes: {response.status_code} {response.text}")
        return None

    resumes = response.json()
    if "items" in resumes:
        return pd.DataFrame(resumes["items"])
    else:
        print("No items found in response.")
        return None

# Example usage
auth_token = "YOUR_AUTH_TOKEN_HERE"  # Replace with your actual OAuth token
data = fetch_resumes("Data Scientist", 1, per_page=50, auth_token=auth_token)

if data is not None:
    data.to_csv("resumes.csv", index=False)
    print("Data saved to resumes.csv")
else:
    print("No data to save.")



Error fetching resumes: 400 {"description":"Bad User-Agent header","errors":[{"value":"blacklisted","type":"bad_user_agent"}],"request_id":"1735038308687ccf09dbb1d9852511b8"}
No data to save.


In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import tensorflow as tf
import pandas as pd

# Load dataset
data = pd.read_csv("resumes.csv")

# Preprocessing
le_position = LabelEncoder()
data['position_encoded'] = le_position.fit_transform(data['position'])

scaler = StandardScaler()
data['experience_scaled'] = scaler.fit_transform(data[['experience']])

# Features and target
X = data[['position_encoded', 'experience_scaled', 'age', 'education_level']]
y = data['is_suitable']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Neural network
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))


FileNotFoundError: [Errno 2] No such file or directory: 'resumes.csv'

In [7]:
import streamlit as st
import pandas as pd
import requests

st.title(\"Сервис подбора персонала\")

uploaded_file = st.file_uploader(\"Загрузите резюме кандидата (CSV)\", type=[\"csv\"])

if uploaded_file:
    resumes = pd.read_csv(uploaded_file)
    st.write(\"Загруженные резюме:\")
    st.write(resumes.head())
    
    vacancy = st.text_input(\"Введите название вакансии:\")
    if st.button(\"Найти подходящих кандидатов\") and vacancy:
        st.write(\"Ищем подходящих кандидатов...\")
        
        # Пример вызова API для поиска кандидатов
        candidates = fetch_resumes(vacancy, 1, per_page=10)
        st.write(\"Подходящие кандидаты:\")
        st.write(candidates)


SyntaxError: unexpected character after line continuation character (<ipython-input-7-cd6aebd29b79>, line 5)